# PARTE A: generación del dataset principal
## Información hogares
### Grupo  13

Importación de librerías necesarias y funciones necesarias para el procesamiento del dataset

In [1]:
import os  
from openpyxl import load_workbook
from pathlib import Path
import csv
import sys

sys.path.append("..")
from src.utils.procesamiento_dataset import obtener_entero, procesar_tipo_hogar, procesar_material_techumbre, procesar_densidad_hogar, procesar_condicion_habitabilidad

Importación de rutas

In [2]:
carpeta = Path("../files") 
archivos_txt = [f for f in os.listdir(carpeta) if "hogar" in f and f.endswith(".txt")] 

hogares = [] 

**Creación de un dataset de hogares a partir de la información disponible**. 

Este dataset contiene además columnas con formato texto que permiten una lectura más clara de los datos. 

•	**TIPO_HOGAR**: indica el tipo de hogar según la cantidad de personas que viven.  

•	**MATERIAL_TECHUMBRE**: indica el tipo de hogar según el material del techo de la vivienda. 

•	**DENSIDAD_HOGAR**: indica la densidad del hogar.

•	**CONDICION_DE_HABITABILIDAD**: califica a las viviendas en Insuficiente, Regular, Saludable y Buenas según las siguientes variables:

        - IV6: tiene agua
        - IV7: origen del agua
        - IV8: posee baño
        - IV9: ubicación del baño
        - IV10: tipo de baño
        - IV11: desague del baño 



In [3]:
for archivo in archivos_txt:
    ruta_archivo = carpeta / archivo
    with open(ruta_archivo, encoding="utf-8") as f:
        lector = csv.reader(f, delimiter=';')
        filas = list(lector)
        
        encabezado = [columna.strip('"') for columna in filas[0]] 
       
    
        try:
            ix_tot_idx = encabezado.index("IX_TOT")      
            iv4_idx = encabezado.index("IV4")     
            iv2_idx = encabezado.index("IV2")
            iv6_idx = encabezado.index("IV6")
            iv7_idx = encabezado.index("IV7")
            iv8_idx = encabezado.index("IV8")
            iv9_idx = encabezado.index("IV9")
            iv10_idx = encabezado.index("IV10") 
            iv11_idx = encabezado.index("IV11")
        except ValueError as e:
            print(f"No se encontró alguna columna en {archivo}: {e}")
            continue
    
    
        for fila in filas[1:]:
            hogar = {encabezado[i]: fila[i] for i in range(len(encabezado))}
            
            hogar = {clave: valor for clave, valor in hogar.items() if clave.strip()} 
  
            hogar["TIPO_HOGAR"] = procesar_tipo_hogar (fila, ix_tot_idx)
            hogar["MATERIAL_TECHUMBRE"] = procesar_material_techumbre (fila, iv4_idx)
            hogar["DENSIDAD_HOGAR"] = procesar_densidad_hogar (fila, iv2_idx, ix_tot_idx)
            hogar["CONDICION_DE_HABITABILIDAD"] = procesar_condicion_habitabilidad (fila, iv6_idx, iv7_idx, iv8_idx, iv9_idx, iv10_idx, iv11_idx)
            
            
            
            hogares.append(hogar)


        

Genero la ruta de salida y archivo csv

In [4]:
ruta_salida_csv = Path("../data_out") / "salida_hogares.csv"


if hogares:
    
    if isinstance(hogares[0], dict):
        campos = [campo for campo in hogares[0].keys() if campo.strip()] 
        
        with open(ruta_salida_csv, "w", newline="", encoding="utf-8") as archivo_csv:
         
            escritor = csv.DictWriter(archivo_csv, fieldnames=campos, delimiter=';')
            escritor.writeheader()
            escritor.writerows(hogares) 
        print(f"Archivo guardado exitosamente como: {ruta_salida_csv}")


Archivo guardado exitosamente como: ../data_out/salida_hogares.csv
